## 📥 1. Importación y Carga de los 4 Dataframes Limpios

Cargaremos los 4 dataframes desde los notebooks ya ejecutados. Esto requiere que los notebooks anteriores hayan sido ejecutados y los dataframes estén disponibles.

In [26]:

# Importaciones necesarias
import os
from pathlib import Path

import numpy as np
import pandas as pd

# Opciones de pandas para debugging
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)

print('Librerias cargadas correctamente.')


Librerias cargadas correctamente.


In [27]:

# Configuracion de rutas y artefactos procesados (ETL)
PROJECT_DIR = Path.cwd()
PATH_PROCESSED = PROJECT_DIR / 'processed'
PATH_PROCESSED.mkdir(parents=True, exist_ok=True)

BASE_FINAL_PATH = PROJECT_DIR / 'db' / 'final' / 'Base_Final_Aurelion.csv'

DATA_SPECS = {
    'df_clientes_True': {
        'label': 'Clientes',
        'filename': 'clientes_limpio.csv',
        'parse_dates': ['fecha_alta'],
    },
    'df_ventas_True': {
        'label': 'Ventas (cabecera)',
        'filename': 'ventas_limpio.csv',
        'parse_dates': ['fecha'],
    },
    'df_detalle_ventas_True': {
        'label': 'Detalle de ventas',
        'filename': 'detalle_ventas_limpio.csv',
        'parse_dates': [],
    },
    'df_productos_True': {
        'label': 'Productos',
        'filename': 'productos_limpio.csv',
        'parse_dates': [],
    },
}

def load_clean_dataframe(var_name, spec):
    file_path = PATH_PROCESSED / spec['filename']
    if not file_path.exists():
        raise FileNotFoundError(f"No se encontro el archivo esperado: {file_path}")
    df = pd.read_csv(file_path, parse_dates=spec.get('parse_dates') or [])
    print(f"[OK ] {spec['label']:<24} -> {df.shape} | fuente: {file_path.relative_to(PROJECT_DIR)}")
    return df

loaded_frames = {var: load_clean_dataframe(var, spec) for var, spec in DATA_SPECS.items()}
globals().update(loaded_frames)
print("Dataframes limpios disponibles en memoria.")


[OK ] Clientes                 -> (100, 5) | fuente: processed\clientes_limpio.csv
[OK ] Ventas (cabecera)        -> (120, 12) | fuente: processed\ventas_limpio.csv
[OK ] Detalle de ventas        -> (343, 6) | fuente: processed\detalle_ventas_limpio.csv
[OK ] Productos                -> (100, 4) | fuente: processed\productos_limpio.csv
Dataframes limpios disponibles en memoria.


In [28]:
# Funcion para verificar y reportar disponibilidad de dataframes

def check_dataframes():
    missing = []
    for var_name, spec in DATA_SPECS.items():
        processed_path = PATH_PROCESSED / spec['filename']
        artifact_status = 'OK' if processed_path.exists() else 'PENDIENTE'
        if var_name in globals():
            df_temp = globals()[var_name]
            print(f"OK  {spec['label']:30s} -> {df_temp.shape} | archivo: {artifact_status}")
        else:
            missing.append(spec['label'])
            print(f"X   {spec['label']:30s} -> NO EN MEMORIA | archivo: {artifact_status}")
    return missing

missing_specs = check_dataframes()
all_available = len(missing_specs) == 0

if all_available:
    print("Todos los dataframes limpios están en memoria.")
    for idx, (var_name, spec) in enumerate(DATA_SPECS.items(), start=1):
        df_temp = globals()[var_name]
        print(f"{idx}. {var_name} -> {spec['label']}")
        print('-'*80)
        print(df_temp.head(0))
        print(f"Columnas: {list(df_temp.columns)}")
        print(f"Registros: {len(df_temp)}")
else:
    print(f" No es posible avanzar. Dataframes faltantes: {missing_specs}")

OK  Clientes                       -> (100, 5) | archivo: OK
OK  Ventas (cabecera)              -> (120, 12) | archivo: OK
OK  Detalle de ventas              -> (343, 6) | archivo: OK
OK  Productos                      -> (100, 4) | archivo: OK
Todos los dataframes limpios están en memoria.
1. df_clientes_True -> Clientes
--------------------------------------------------------------------------------
Empty DataFrame
Columns: [id_cliente, nombre_cliente, email, ciudad, fecha_alta]
Index: []
Columnas: ['id_cliente', 'nombre_cliente', 'email', 'ciudad', 'fecha_alta']
Registros: 100
2. df_ventas_True -> Ventas (cabecera)
--------------------------------------------------------------------------------
Empty DataFrame
Columns: [id_venta, fecha, id_cliente, medio_pago_efectivo, medio_pago_qr, medio_pago_tarjeta, medio_pago_transferencia, anio, mes, dia_semana, trimestre, transacciones_cliente]
Index: []
Columnas: ['id_venta', 'fecha', 'id_cliente', 'medio_pago_efectivo', 'medio_pago_qr', 'me

## 🔗 3. Preparación: Renombrado de Claves Foráneas

En la tabla de detalle de ventas, si existe un nombre diferente para la FK de producto (ej: FK_producto), lo renombramos a `id_producto` para que el merge sea unívoco.

In [29]:
if all_available:
    print('='*100)
    print('🔧 PREPARACIÓN: Validación y renombrado de columnas')
    print('='*100)
    
    # Verificar y renombrar columna de producto en detalle_ventas si es necesario
    if 'FK_producto' in df_detalle_ventas_True.columns:
        print('\n✓ Renombrando FK_producto → id_producto en df_detalle_ventas_True')
        df_detalle_ventas_True.rename(columns={'FK_producto': 'id_producto'}, inplace=True)
    elif 'id_producto' not in df_detalle_ventas_True.columns:
        print('\n⚠️  Aviso: No se encontró FK_producto ni id_producto en detalle_ventas')
        print(f'   Columnas disponibles: {list(df_detalle_ventas_True.columns)}')
    else:
        print('\n✓ Columna id_producto ya existe en df_detalle_ventas_True')
    
    # Validar que todos los FK referencias existan en las PK correspondientes
    print('\n' + '-'*80)
    print('📌 Validación de Integridad Referencial:')
    print('-'*80)
    
    # FK id_cliente en ventas
    clientes_en_ventas = set(df_ventas_True['id_cliente'].dropna().unique())
    clientes_existentes = set(df_clientes_True['id_cliente'].unique())
    clientes_huerfanos = clientes_en_ventas - clientes_existentes
    print(f'\n✓ FK id_cliente en ventas:',)
    if not clientes_huerfanos:
        print(f'OK ({len(clientes_en_ventas)} referencias válidas)')
    else:
        print(f'INCONSISTENCIA: {len(clientes_huerfanos)} huérfanos')
    
    # FK id_venta en detalle
    ventas_en_detalle = set(df_detalle_ventas_True['id_venta'].dropna().unique())
    ventas_existentes = set(df_ventas_True['id_venta'].unique())
    ventas_huerfanas = ventas_en_detalle - ventas_existentes
    print(f'✓ FK id_venta en detalle:',)
    if not ventas_huerfanas:
        print(f'OK ({len(ventas_en_detalle)} referencias válidas)')
    else:
        print(f'INCONSISTENCIA: {len(ventas_huerfanas)} huérfanos')
    
    # FK id_producto en detalle
    productos_en_detalle = set(df_detalle_ventas_True['id_producto'].dropna().unique())
    productos_existentes = set(df_productos_True['id_producto'].unique())
    productos_huerfanos = productos_en_detalle - productos_existentes
    print(f'✓ FK id_producto en detalle:',)
    if not productos_huerfanos:
        print(f'OK ({len(productos_en_detalle)} referencias válidas)')
    else:
        print(f'INCONSISTENCIA: {len(productos_huerfanos)} huérfanos')

🔧 PREPARACIÓN: Validación y renombrado de columnas

✓ Columna id_producto ya existe en df_detalle_ventas_True

--------------------------------------------------------------------------------
📌 Validación de Integridad Referencial:
--------------------------------------------------------------------------------

✓ FK id_cliente en ventas:
OK (67 referencias válidas)
✓ FK id_venta en detalle:
OK (120 referencias válidas)
✓ FK id_producto en detalle:
OK (95 referencias válidas)


## 🔀 4. Merge de Tablas - Consolidación de Base Final

Realizamos los merges en orden siguiendo la estructura de relaciones:
1. `df_clientes_True` ← merge ← `df_ventas_True` (on=id_cliente)
2. Resultado ← merge ← `df_detalle_ventas_True` (on=id_venta)
3. Resultado ← merge ← `df_productos_True` (on=id_producto)

In [30]:

if all_available:
    print('='*100)
    print(' MERGE - Consolidacion de Base Final')
    print('='*100)

    detalle_cols = list(df_detalle_ventas_True.columns)
    prefer_prod = ['id_producto', 'nombre_producto', 'categoria', 'marca', 'subcategoria', 'tipo_producto']
    cols_prod = []
    for col in prefer_prod:
        if col == 'id_producto' and col in df_productos_True.columns:
            if col not in cols_prod:
                cols_prod.append(col)
        elif col in df_productos_True.columns and col not in detalle_cols:
            cols_prod.append(col)
    if 'id_producto' not in cols_prod and 'id_producto' in df_productos_True.columns:
        cols_prod.insert(0, 'id_producto')
    if len(cols_prod) == 1:
        extras = [c for c in df_productos_True.columns if c not in detalle_cols and c != 'id_producto']
        cols_prod.extend(extras)
    df_productos_dim = df_productos_True[cols_prod].copy()
    print(f"Columnas agregadas desde productos: {cols_prod}")

    df_detalle_prod = df_detalle_ventas_True.merge(
        df_productos_dim,
        on='id_producto',
        how='left',
        validate='m:1'
    )
    print(f"Dimensiones detalle+productos: {df_detalle_prod.shape}")

    ventas_cols = list(df_ventas_True.columns)
    prefer_cli = ['id_cliente', 'nombre_cliente', 'segmento', 'provincia', 'ciudad', 'region', 'pais', 'categoria_cliente']
    cols_cli = []
    for col in prefer_cli:
        if col == 'id_cliente' and col in df_clientes_True.columns:
            if col not in cols_cli:
                cols_cli.append(col)
        elif col in df_clientes_True.columns and col not in ventas_cols:
            cols_cli.append(col)
    if 'id_cliente' not in cols_cli and 'id_cliente' in df_clientes_True.columns:
        cols_cli.insert(0, 'id_cliente')
    if len(cols_cli) == 1:
        extras_cli = [c for c in df_clientes_True.columns if c not in ventas_cols and c != 'id_cliente']
        cols_cli.extend(extras_cli)
    df_clientes_dim = df_clientes_True[cols_cli].copy()
    print(f"Columnas agregadas desde clientes: {cols_cli}")

    df_ventas_cli = df_ventas_True.merge(
        df_clientes_dim,
        on='id_cliente',
        how='left',
        validate='m:1'
    )
    print(f"Dimensiones ventas+clientes: {df_ventas_cli.shape}")

    df_final = df_detalle_prod.merge(
        df_ventas_cli,
        on='id_venta',
        how='left',
        validate='m:1',
        suffixes=('', '_ventas')
    )
    print(f"Dimensiones finales tras merge: {df_final.shape}")

    duplicated_cols = df_final.columns[df_final.columns.duplicated()]
    print('Columnas duplicadas detectadas:', duplicated_cols.tolist())
    if duplicated_cols.any():
        df_final = df_final.loc[:, ~df_final.columns.duplicated()]
        print('Columnas duplicadas eliminadas tras validar equivalencia.')

    base_consolidada = df_final.copy()
    print('\nMerge completado exitosamente!')


 MERGE - Consolidacion de Base Final
Columnas agregadas desde productos: ['id_producto', 'nombre_producto', 'categoria']
Dimensiones detalle+productos: (343, 8)
Columnas agregadas desde clientes: ['id_cliente', 'nombre_cliente', 'ciudad']
Dimensiones ventas+clientes: (120, 14)
Dimensiones finales tras merge: (343, 21)
Columnas duplicadas detectadas: []

Merge completado exitosamente!

 MERGE - Consolidacion de Base Final
Columnas agregadas desde productos: ['id_producto', 'nombre_producto', 'categoria']
Dimensiones detalle+productos: (343, 8)
Columnas agregadas desde clientes: ['id_cliente', 'nombre_cliente', 'ciudad']
Dimensiones ventas+clientes: (120, 14)
Dimensiones finales tras merge: (343, 21)
Columnas duplicadas detectadas: []

Merge completado exitosamente!


## 📊 5. Inspección de la Base Consolidada

In [31]:
if 'base_consolidada' in locals():
    print('='*100)
    print('📋 INSPECCIÓN - Base Consolidada')
    print('='*100)
    
    print('\nPrimeras 5 filas:')
    print('-'*80)
    print(base_consolidada.head())
    
    print('\n\nÚltimas 5 filas:')
    print('-'*80)
    print(base_consolidada.tail())
    
    print('\n\nTipos de datos:')
    print('-'*80)
    print(base_consolidada.dtypes)
    
    print('\n\nValores nulos por columna:')
    print('-'*80)
    nulls = base_consolidada.isnull().sum()
    if nulls.sum() == 0:
        print('✓ NO hay valores nulos en la base consolidada')
    else:
        print(nulls[nulls > 0])
    
    print('\n\nEstadísticas generales:')
    print('-'*80)
    print(f'Dimensiones: {base_consolidada.shape}')
    print(f'Memoria usada: {base_consolidada.memory_usage(deep=True).sum() / (1024**2):.2f} MB')
    print(f'Clientes únicos: {base_consolidada["id_cliente"].nunique()}')
    print(f'Ventas (id_venta) únicas: {base_consolidada["id_venta"].nunique()}')
    print(f'Detalles de venta únicos: {base_consolidada["id_detalle"].nunique() if "id_detalle" in base_consolidada.columns else "N/A"}')
    print(f'Productos únicos: {base_consolidada["id_producto"].nunique()}')

📋 INSPECCIÓN - Base Consolidada

Primeras 5 filas:
--------------------------------------------------------------------------------
   id_venta  id_producto  cantidad  precio_unitario  importe  importe_std        nombre_producto  categoria  \
0         1           90         1           2902.0   2902.0    -0.918259    Toallas Húmedas X50  alimentos   
1         2           82         5           2394.0  11970.0     0.806396  Aceitunas Negras 200G  alimentos   
2         2           39         5            469.0   2345.0    -1.024196     Helado Vainilla 1L  alimentos   
3         2           70         2           4061.0   8122.0     0.074540           Fernet 750Ml  alimentos   
4         2           22         1           2069.0   2069.0    -1.076688  Medialunas De Manteca  alimentos   

       fecha  id_cliente  medio_pago_efectivo  medio_pago_qr  medio_pago_tarjeta  medio_pago_transferencia  anio  mes  \
0 2024-06-19          62                  0.0            0.0                 1.0

## 🧹 6. Preprocesamiento y Limpieza Adicional

En esta sección aplicamos transformaciones finales para garantizar que la base esté lista para análisis.

In [32]:
if 'base_consolidada' in locals():
    print('='*100)
    print('🧹 PREPROCESAMIENTO - Limpieza y Transformaciones Finales')
    print('='*100)
    
    # Crear copia para no modificar original hasta validar
    base_final = base_consolidada.copy()
    
    # 1. Detectar y manejar duplicados
    print('\n1️⃣  Detección de duplicados')
    print('-'*80)
    dup_count = base_final.duplicated().sum()
    print(f'Filas duplicadas: {dup_count}')
    if dup_count > 0:
        print(f'  (Eliminando {dup_count} filas duplicadas)')
        base_final = base_final.drop_duplicates()
        print(f'  Nuevo tamaño: {base_final.shape}')
    
    # 2. Renombrar columnas para mayor claridad y evitar colisiones
    print('\n2️⃣  Estandarización de nombres de columnas')
    print('-'*80)
    rename_map = {}
    for col in base_final.columns:
        # Aquí puedes definir transformaciones de nombres si es necesario
        # Ejemplo: renombrar columnas que tengan sufijos _x o _y
        if col.endswith('_x'):
            rename_map[col] = col[:-2]
        elif col.endswith('_y'):
            rename_map[col] = col[:-2]
    
    if rename_map:
        print(f'Columnas renombradas:')
        for old, new in rename_map.items():
            print(f'  {old:30s} → {new}')
        base_final.rename(columns=rename_map, inplace=True)
    else:
        print('✓ No hay columnas con sufijos _x o _y')
    

    # 2b. Eliminar columnas duplicadas post-merge
    print('\n2b  Columnas duplicadas tras el merge')
    print('-'*80)
    duplicated_mask = base_final.columns.duplicated()
    if duplicated_mask.any():
        dup_names = base_final.columns[duplicated_mask]
        dup_list = sorted({str(name) for name in dup_names})
        base_final = base_final.loc[:, ~duplicated_mask]
        print(f" Columnas eliminadas: {dup_list}")
    else:
        print(' No se detectaron columnas duplicadas')

    # 3. Reordenar columnas de forma lógica (primero IDs, luego datos descriptivos)
    print('\n3️⃣  Reorganización de columnas (orden lógico)')
    print('-'*80)
    id_cols = [c for c in base_final.columns if 'id' in c.lower()]
    date_cols = [c for c in base_final.columns if 'fecha' in c.lower() or 'date' in c.lower()]
    money_cols = [c for c in base_final.columns if any(x in c.lower() for x in ['precio', 'importe', 'monto', 'total'])]
    other_cols = [c for c in base_final.columns if c not in id_cols + date_cols + money_cols]
    
    new_order = id_cols + date_cols + other_cols + money_cols
    base_final = base_final[[c for c in new_order if c in base_final.columns]]
    print(f'✓ Columnas reordenadas: {len(new_order)} columnas')
    print(f'  - IDs (PK/FK): {len(id_cols)}')
    print(f'  - Fechas: {len(date_cols)}')
    print(f'  - Dinero (precios/importes): {len(money_cols)}')
    print(f'  - Otros: {len(other_cols)}')
    
    # 4. Tipos de datos finales
    print('\n4️⃣  Validación de tipos de datos')
    print('-'*80)
    print(base_final.dtypes)
    
    print('\n✨ Preprocesamiento completado')

🧹 PREPROCESAMIENTO - Limpieza y Transformaciones Finales

1️⃣  Detección de duplicados
--------------------------------------------------------------------------------
Filas duplicadas: 0

2️⃣  Estandarización de nombres de columnas
--------------------------------------------------------------------------------
✓ No hay columnas con sufijos _x o _y

2b  Columnas duplicadas tras el merge
--------------------------------------------------------------------------------
 No se detectaron columnas duplicadas

3️⃣  Reorganización de columnas (orden lógico)
--------------------------------------------------------------------------------
✓ Columnas reordenadas: 21 columnas
  - IDs (PK/FK): 4
  - Fechas: 1
  - Dinero (precios/importes): 3
  - Otros: 13

4️⃣  Validación de tipos de datos
--------------------------------------------------------------------------------
id_venta                             int64
id_producto                          int64
cantidad                             int64


## 📄 7. Exportación a CSV

In [33]:
if 'base_final' in locals():
    print('='*100)
    print(' EXPORTACION - Base Final a CSV')
    print('='*100)

    print("\n--- Info de Base_Final_Aurelion (previa a exportación) ---")
    # Forzar a que muestre todas las columnas
    base_final.info(verbose=True)

    BASE_FINAL_PATH.parent.mkdir(parents=True, exist_ok=True)
    print(f"\nGuardando Base_Final_Aurelion en: {BASE_FINAL_PATH}")
    base_final.to_csv(BASE_FINAL_PATH, index=False)
    print(f"Shape: {base_final.shape} | Columnas: {len(base_final.columns)}")

    import hashlib

    def compute_md5(path_to_file):
        hash_md5 = hashlib.md5()
        with open(path_to_file, 'rb') as file:
            for chunk in iter(lambda: file.read(8192), b''):
                hash_md5.update(chunk)
        return hash_md5.hexdigest()

    md5_value = compute_md5(BASE_FINAL_PATH)
    md5_file = BASE_FINAL_PATH.with_suffix('.md5')
    md5_file.write_text(md5_value)
    print(f"Checksum MD5 ({md5_value}) guardado en: {md5_file}")

    if BASE_FINAL_PATH.exists():
        df_base = pd.read_csv(BASE_FINAL_PATH, parse_dates=['fecha'])
        print("\n--- Estructura de Base_Final_Aurelion (desde CSV) ---")
        # Igual acá, para ver todas las columnas
        df_base.info(verbose=True)

        cols_duplicadas_base = df_base.columns[df_base.columns.duplicated()]
        print('Columnas duplicadas en Base_Final_Aurelion:', cols_duplicadas_base.tolist())
        try:
            pd.testing.assert_frame_equal(
                base_final.reset_index(drop=True),
                df_base.reset_index(drop=True),
                check_dtype=False,
                atol=1e-9,
                rtol=1e-9
            )
            print('El dataframe en memoria coincide con el CSV exportado.')
        except AssertionError as err:
            print('Diferencias detectadas entre base_final y el CSV exportado.')
            print(err)
    else:
        print('Error: El archivo no se creó correctamente')

    print('Consolidación completada exitosamente')
else:
    print('No se encontró base_final en memoria. Revise los pasos anteriores.')


 EXPORTACION - Base Final a CSV

--- Info de Base_Final_Aurelion (previa a exportación) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343 entries, 0 to 342
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   id_venta                  343 non-null    int64         
 1   id_producto               343 non-null    int64         
 2   cantidad                  343 non-null    int64         
 3   id_cliente                343 non-null    int64         
 4   fecha                     343 non-null    datetime64[ns]
 5   nombre_producto           343 non-null    object        
 6   categoria                 343 non-null    object        
 7   medio_pago_efectivo       343 non-null    float64       
 8   medio_pago_qr             343 non-null    float64       
 9   medio_pago_tarjeta        343 non-null    float64       
 10  medio_pago_transferencia  343 non-null    float64     

## 📋 8. Resumen Final

In [34]:

print('='*100)
print(' RESUMEN FINAL - CONSOLIDACION PROYECTO AURELION')
print('='*100)

print(' Objetivo cumplido:')
print('    Se integraron los 4 dataframes limpios (clientes, ventas, detalle_ventas, productos)')
print('    Se aplicaron relaciones de clave primaria (PK) y foranea (FK)')
print('    Se valido integridad referencial')
print('    Se realizo preprocesamiento y limpieza adicional')
print('    Se exporto base consolidada a CSV')

print(' Estadisticas finales:')
if 'base_final' in locals():
    print(f"   - Registros en base final: {len(base_final):,}")
    print(f"   - Columnas integradas: {len(base_final.columns)}")
    print(f"   - Memoria usada: {base_final.memory_usage(deep=True).sum() / (1024**2):.2f} MB")
    print(f"   - Ubicacion archivo CSV: {BASE_FINAL_PATH}")

print(' Proximos pasos:')
print('   1. Usar Base_Final_Aurelion.csv para analisis avanzados')
print('   2. Crear visualizaciones estrategicas')
print('   3. Desarrollar modelos predictivos o de segmentacion')
print('   4. Generar reportes ejecutivos')

print(' Consolidacion finalizada correctamente')


 RESUMEN FINAL - CONSOLIDACION PROYECTO AURELION
 Objetivo cumplido:
    Se integraron los 4 dataframes limpios (clientes, ventas, detalle_ventas, productos)
    Se aplicaron relaciones de clave primaria (PK) y foranea (FK)
    Se valido integridad referencial
    Se realizo preprocesamiento y limpieza adicional
    Se exporto base consolidada a CSV
 Estadisticas finales:
   - Registros en base final: 343
   - Columnas integradas: 21
   - Memoria usada: 0.14 MB
   - Ubicacion archivo CSV: c:\Users\Asus\Desktop\Augusto Villegas - Proyecto Aurelion\db\final\Base_Final_Aurelion.csv
 Proximos pasos:
   1. Usar Base_Final_Aurelion.csv para analisis avanzados
   2. Crear visualizaciones estrategicas
   3. Desarrollar modelos predictivos o de segmentacion
   4. Generar reportes ejecutivos
 Consolidacion finalizada correctamente
